In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from numpy.linalg import inv
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score
import nltk
import json
import string
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df_pos = pd.read_csv('/content/drive/MyDrive/ENGI 4800/data/Software_Matched_Sentences.csv', index_col=False)[['Original_txt','Stemmed','Label']]
df_pos['Label'] = df_pos['Label'].fillna(1).astype('int')
df_pos['Label'][1] = 1
df_neg = pd.read_csv('/content/drive/MyDrive/ENGI 4800/data/Software_Unmatched_Sentences.csv', index_col=False)[['Original_txt', 'Stemmed','Label']]
df_neg['Label'] = df_neg['Label'].fillna(0).astype('int')
df = pd.concat([df_pos, df_neg])
m, n = df.shape
df.index = np.arange(m)

UnicodeDecodeError: ignored

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Original_txt  205 non-null    object
 1   Stemmed       205 non-null    object
 2   Label         205 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 6.4+ KB


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
sbert_embeddings = model.encode(df['Stemmed'])
sbert_embeddings.shape

(205, 768)

In [ ]:
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.linear_model import LogisticRegression
label = np.array(df['Label'])
#logistic regression model
lr = LogisticRegression(random_state = 42)
sbert_cv = cross_validate(lr, sbert_embeddings, label, cv=5,\
                     scoring=('accuracy', 'f1', 'roc_auc'),\
                     groups = label,\
                     return_train_score=True,\
                     return_estimator = True)

In [ ]:
print('Tfidf training accuracy {}, f1-score {}, auc score {}'.format(np.mean(sbert_cv['train_accuracy']),\
                                                                     np.mean(sbert_cv['train_f1']),\
                                                                     np.mean(sbert_cv['train_roc_auc'])))
                                                                        
      
print('Tfidf testing accuracy {}, f1-score {}, auc score {}'.format(np.mean(sbert_cv['test_accuracy']),\
                                                                    np.mean(sbert_cv['test_f1']),\
                                                                    np.mean(sbert_cv['test_roc_auc'])))

Tfidf training accuracy 1.0, f1-score 1.0, auc score 1.0
Tfidf testing accuracy 0.975609756097561, f1-score 0.9735003295978906, auc score 0.9961745272271589


In [ ]:
lr = LogisticRegression(random_state = 42)
sbert_cv_label = cross_val_predict(lr, sbert_embeddings, label, cv=5,\
                     groups = label)
sum(sbert_cv_label == label)

200

In [ ]:
# print mislabeled samples
for txt, lab in zip(df.iloc[sbert_cv_label != label,:]['Original_txt'], df.iloc[sbert_cv_label != label,:]['Label']):
  print(lab, txt)

1  #57207248035  and  #56612620200  proposed the research point, conducted the program coding and paper writing
1 The data was cleaned and coded by  #57222091689  and  #57210805253 B., who also initially helped in the adaptation of the coding manual
1 #56457888300  wrote the code, performed all the analysis, and produced all the figures
0  #14007670200 : performed cell culture, performed biologic assays, conducted Western blot analysis, planed the study, analyzed the data, wrote the draft
0 #7404297635 : Visualization, Investigation, Writing - Review & Editing  #55270770100 : Supervision, Resources


In [ ]:
import spacy
from spacy import displacy

# Pos tagging with spacy
nlp = spacy.load("en_core_web_sm")
# An example
doc = nlp("John and Jane proposed the research point, conducted the program coding and paper writing")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)
displacy.render(doc, style="dep", jupyter=True)

John John PROPN NNP nsubj Xxxx True False
and and CCONJ CC cc xxx True True
Jane Jane PROPN NNP conj Xxxx True False
proposed propose VERB VBD ROOT xxxx True False
the the DET DT det xxx True True
research research NOUN NN compound xxxx True False
point point NOUN NN dobj xxxx True False
, , PUNCT , punct , False False
conducted conduct VERB VBD conj xxxx True False
the the DET DT det xxx True True
program program NOUN NN compound xxxx True False
coding coding NOUN NN dobj xxxx True False
and and CCONJ CC cc xxx True True
paper paper NOUN NN compound xxxx True False
writing writing NOUN NN conj xxxx True False


In [ ]:
# Another example
doc = nlp("John : performed cell culture, performed biologic assays, conducted Western blot analysis, planed the study, analyzed the data, wrote the draft")

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)
displacy.render(doc, style="dep", jupyter=True)

John John PROPN NNP nsubj Xxxx True False
: : PUNCT : punct : False False
performed perform VERB VBD ROOT xxxx True False
cell cell NOUN NN compound xxxx True False
culture culture NOUN NN dobj xxxx True False
, , PUNCT , punct , False False
performed perform VERB VBD advcl xxxx True False
biologic biologic ADJ JJ amod xxxx True False
assays assay NOUN NNS dobj xxxx True False
, , PUNCT , punct , False False
conducted conduct VERB VBN amod xxxx True False
Western western ADJ JJ amod Xxxxx True False
blot blot NOUN NN compound xxxx True False
analysis analysis NOUN NN dobj xxxx True False
, , PUNCT , punct , False False
planed plane VERB VBD conj xxxx True False
the the DET DT det xxx True True
study study NOUN NN dobj xxxx True False
, , PUNCT , punct , False False
analyzed analyze VERB VBD conj xxxx True False
the the DET DT det xxx True True
data data NOUN NN dobj xxxx True False
, , PUNCT , punct , False False
wrote write VERB VBD conj xxxx True False
the the DET DT det xxx True Tru

## Classifer with cosine similarities

In [ ]:
import math
def cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

In [ ]:
import string
punctuations = string.punctuation
txt_lis = []
for txt in df['Original_txt']: 
    tmp_lis = []
    doc = nlp(txt.lower())
    for token in doc:
        # Only keep words with pos taggings in ['VERB', 'NOUN', 'ADJ', 'ADV', 'PROPN']
        if (token.text == ',') or (token.text == 'and')  or (token.pos_ in ['VERB', 'NOUN', 'ADJ', 'ADV', 'PROPN']):
            tmp_lis.append(token.text.strip())
    txt_lis.append(' '.join(tmp_lis))

In [ ]:
# Befeore processing 
print(df['Original_txt'][0])
# After processing
print(txt_lis[0])

#57239940500 : Data curation, Software, Formal analysis, Writing -review & editing
data curation , software , formal analysis , writing -review editing


In [ ]:
# Further parsing
fin_lis = []
for txt in txt_lis:
    tmp_lis = []
    # Split sentences by 'and' and ','
    for i in txt.split(','):
        for j in i.split('and'):
            tmp_lis.append(j.strip())
    fin_lis.append(tmp_lis)

In [ ]:
print(df['Original_txt'][0])
print(fin_lis[0])
print('---------------------------')
print(df['Original_txt'][7])
print(fin_lis[7])

#57239940500 : Data curation, Software, Formal analysis, Writing -review & editing
['data curation', 'software', 'formal analysis', 'writing -review editing']
---------------------------
#7004297142  codeveloped and edited the manuscript.
['codeveloped', 'edited manuscript']


In [ ]:
threshold = 0.766 # A ramdom threshold
acc = []

# In preprocessing, we split sentences into phrases like ['codeveloped', 'edited manuscript']
# Idea is that for a target label software, we compute the distance between the embedding of phrase and embedding of 'software'.
# (We can also do this with n gram)
# If the similarity reachs threshold, we assume this sentence belongs to software class.
gen_label = []
software_embedding = model.encode('software')
for i in range(len(fin_lis)):
    l = 0
    tmp_lis = fin_lis[i]
    for sen in tmp_lis:
        sen_embedding = model.encode(sen)
        sim = cosine_similarity(sen_embedding, software_embedding)
        if sim > threshold:
            l = 1
            break
        
    gen_label.append(l)
acc.append(sum(np.array(gen_label) == label)/len(label))

In [ ]:
#Accuracy
acc

[0.9219512195121952]

In [ ]:
"""
# Hyperparameters tuning, find best threshold
acc = []
for threshold in np.arange(0.70, 1.00, 0.05):
    gen_label = []
    software_embedding = model.encode('software')
    for i in range(len(fin_lis)):
        l = 0
        tmp_lis = fin_lis[i]
        for sen in tmp_lis:
            sen_embedding = model.encode(sen)
            sim = cosine_similarity(sen_embedding, software_embedding)
            if sim > threshold:
                l = 1
                break
            
        gen_label.append(l)
    acc.append(sum(np.array(gen_label) == label)/len(label))
"""

"\nacc = []\nfor threshold in np.arange(0.70, 1.00, 0.05):\n    gen_label = []\n    software_embedding = model.encode('software')\n    for i in range(len(fin_lis)):\n        l = 0\n        tmp_lis = fin_lis[i]\n        for sen in tmp_lis:\n            sen_embedding = model.encode(sen)\n            sim = cosine_similarity(sen_embedding, software_embedding)\n            if sim > threshold:\n                l = 1\n                break\n            \n        gen_label.append(l)\n    acc.append(sum(np.array(gen_label) == label)/len(label))\n"

In [ ]:
#Accuracy
acc

[0.9219512195121952]